# Siting First Aid Stations with LSCP on Toronto's University Campus

*Authors:* [Timothy Ellersiek](https://github.com/TimMcCauley)

This notebook implements the location set covering problem (LSCP) to site first aid stations for Toronto's University campus. LSCP determines locations such that as many demand points as possible are within the maximum service radius. By doing so, the number of locations required to cover all demand points is minimized. In contrast to the maximum covering location problem it does not require a maximum amount of locations as an input parameter. Hence, the use cases for LSCP are specific. One common scenario is finding suitable locations for hospitals in cities where the amount is governed by reachability preconditions to households in a certain amount of time.  

In this notebook demand is described as buildings within Toronto's University campus. Possible locations for the first aid stations are determined using [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) which fetches geometries directly from [OpenStreetMap.org](https://openstreetmap.org).

The routing matrix is computed using [Open Source Routing Machine's demo server API](http://project-osrm.org).

## Prerequisites

To run this jupyter notebook locally, you will have to install the following additional libraries into your python environment.

- [folium](https://python-visualization.github.io/folium)
- [overpy](https://github.com/DinoTools/python-overpy)
- [routing-py](https://github.com/gis-ops/routing-py)
- [shapely](https://github.com/shapely/shapely)

In [1]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

Last updated: 2022-10-19T11:20:06.819903-04:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

Compiler    : Clang 13.0.1 
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [2]:
import folium
import json
import numpy
import overpy
import pulp
import routingpy
from routingpy import OSRM
import shapely
from shapely.geometry import shape, Point
import spopt
from spopt.locate.coverage import LSCP

%watermark -w
%watermark -iv

Watermark: 2.3.1

routingpy: 1.1.0
shapely  : 1.8.5
spopt    : 0.4.1+200.g336f235.dirty
folium   : 0.12.1.post1
json     : 2.0.9
overpy   : 0.6
pulp     : 2.6.0
numpy    : 1.23.3



--------------------------

Create `overpy.Overpass()` instance in order to make queries

In [3]:
api = overpy.Overpass()

First of all we want to fetch all buildings in the University of Toronto boundary

In [4]:
result = api.query(
    """
    [out:json];
    
    area[name="Toronto"]->.b;
    way(area.b)[name="University of Toronto"];
    map_to_area -> .a;
    
    way[building="university"](area.a);
    out center;
    """
)

Create a `folium.Map()` instance to visualize the data

In [5]:
m = folium.Map(
    location=[43.6624674, -79.3988052], tiles="cartodbpositron", zoom_start=15
)

Let's add our results to the map!

In [6]:
university_buildings = []
university_buildings_fg = folium.FeatureGroup("University Buildings").add_to(m)
for idx, way in enumerate(result.ways):
    folium.CircleMarker(
        location=[way.center_lat, way.center_lon],
        radius=3,
        fill=True,
        fill_opacity=0.3,
        popup=folium.Popup(way.tags.get("name", "n/a"), show=False),
        color="blue",
    ).add_to(university_buildings_fg)
    # building centroids
    university_buildings.append(
        {
            "name": way.tags.get("name", "n/a"),
            "center": [way.center_lon, way.center_lat],
            "idx": idx,
        }
    )
m

And now we want to fetch some roads in the same area and return their center points which we can use for our potential locations for first aid stations

In [7]:
result_roads = api.query(
    """
    [out:json];
    
    area[name="Toronto"]->.b;
    way(area.b)[name="University of Toronto"];
    map_to_area -> .a;
    
    way["highway"~"secondary|tertiary|unclassified"](area.a);
    
    out center;
    """
)

Similarly we want to add them to the map

In [8]:
first_aid_stations = []
first_aid_stations_fg = folium.FeatureGroup("First Aid Station Locations").add_to(m)
for idx, way in enumerate(result_roads.ways):
    folium.CircleMarker(
        location=[way.center_lat, way.center_lon],
        radius=2,
        fill=True,
        fill_opacity=1,
        popup=folium.Popup(way.tags.get("name", "n/a"), show=False),
        color="black",
    ).add_to(first_aid_stations_fg)
    # road midpoints
    first_aid_stations.append(
        {
            "name": way.tags.get("name", "n/a"),
            "center": [way.center_lon, way.center_lat],
            "idx": idx+len(university_buildings),
        }
    )
m

## Computing the Distance (Routing) Matrix

Given our potential first aid station locations and the university buildings on the campus we want to compute our distance matrix. Instead of using simple euclidean distance we can make use of proper routes following roads which will yield more realistic travel times. The [routing-py library](https://github.com/gis-ops/routing-py) helps us achieve this which features the ability to use different open source frameworks such as OSRM or Valhalla as well as 3rd party API's such as Google Maps or HERE.

In [9]:
client = OSRM(base_url="https://router.project-osrm.org")

Isolate all location coordinates

In [21]:
university_buildings_locations = [bld["center"] for bld in university_buildings]
first_aid_stations_locations = [fab["center"] for fab in first_aid_stations]
all_locations = university_buildings_locations + first_aid_stations_locations
all_locations[:5]

[[Decimal('-79.3988052'), Decimal('43.6624674')],
 [Decimal('-79.3994442'), Decimal('43.6643897')],
 [Decimal('-79.3895107'), Decimal('43.6648522')],
 [Decimal('-79.3971848'), Decimal('43.6625440')],
 [Decimal('-79.3958563'), Decimal('43.6628344')]]

In [22]:
#first_aid_stations_locations = [fab["center"] for fab in first_aid_stations]
#university_buildings_locations = [bld["center"] for bld in university_buildings]
#all_locations = first_aid_stations_locations + university_buildings_locations
#all_locations[:5]

Define source and target indices

In [23]:
source_indices = [i["idx"] for i in university_buildings]
target_indices = [i["idx"] for i in first_aid_stations]

In [24]:
source_indices[-5:], target_indices[:5]

([114, 115, 116, 117, 118], [119, 120, 121, 122, 123])

In [25]:
#uni_idx2dv = {i["idx"]:i["dv"] for i in university_buildings}
#uni_idx2dv

In [26]:
#fab_idx2dv = {i["idx"]:i["dv"] for i in first_aid_stations}
#fab_idx2dv

Generate routing matrix object

In [27]:
osrm_routing_matrix = client.matrix(
    dry_run=False,
    locations=all_locations,
    sources=source_indices,
    destinations=target_indices,
    profile="pedestrian",
)

Isolate route durations

In [28]:
cost_matrix = numpy.array(osrm_routing_matrix.durations)
cost_matrix.shape

(119, 39)

In [29]:
cost_matrix

array([[240.3, 131.9, 161.8, ...,  68.3,  24.3,  67.5],
       [218.1, 109.7, 153.5, ...,  68.2,  22.8, 117.9],
       [183.4, 136.4, 140.1, ..., 230.6, 190.3, 274.2],
       ...,
       [250.3, 159.2, 111.9, ...,   3.9,  82.1,  63.2],
       [252.1, 144.7, 188.5, ..., 133.5,  88.1, 177.9],
       [181.5,  44.5,  88.3, ..., 150.4, 188.4, 194. ]])

## Solving the LSCP

Last but not least we want to run the solver. We choose the Pulp CBC solver and set the distance cutoff to 2 minutes and 30 seconds afoot. Finally we loop over the solution and add the sited stations and the corresponding allocated buildings to the map with different radiuses.  

In [30]:
solver = pulp.PULP_CBC_CMD(msg=False)  # see solvers available in pulp reference
SERVICE_RADIUS = 150  # maximum acceptable service duration, 2 minutes and 30 seconds

#SERVICE_RADIUS = 200  # maximum acceptable service duration, 2 minutes and 30 seconds

Create and solve an LSCP instance

In [31]:
lscp = LSCP.from_cost_matrix(cost_matrix, SERVICE_RADIUS)
lscp = lscp.solve(solver)

How many first aid sites are needed to entirely cover the 

In [32]:
lscp.problem.objective.value()

3.0

Add the results to the map

In [33]:
sited_first_aid_stations_fg = folium.FeatureGroup("Sited First Aid Stations").add_to(m)
allocated_buildings_demand_fg = folium.FeatureGroup("Allocated Buildings").add_to(m)
solution = {}
colors = [
    "darkcyan",
    "mediumseagreen",
    "saddlebrown",
    "lightskyblue",
    "darkgoldenrod",
    "coral",
    "mediumvioletred",
    "blueviolet",
    "mediumorchid",
]
client_marker_size = 20
numpy.random.seed(0)
for i in range(len(first_aid_stations)):
    if lscp.fac2cli[i]:
        color_choice_idx = numpy.random.choice(range(len(colors)))
        color = colors.pop(color_choice_idx)
        
        # selected candidate facility sites
        folium.CircleMarker(
            location=[
                first_aid_stations[i]["center"][1],
                first_aid_stations[i]["center"][0],
            ],
            radius=30,
            fill=True,
            popup=folium.Popup(f"Sited First Aid Station (index: {i})", show=True),
            color=color,
        ).add_to(sited_first_aid_stations_fg)
        
        # client locations with associated facility symbology
        client_marker_size -= 4
        solution[i] = []
        for j in lscp.fac2cli[i]:
            solution[i].append(university_buildings[j])
            folium.CircleMarker(
                location=[
                    university_buildings[j]["center"][1],
                    university_buildings[j]["center"][0],
                ],
                radius=client_marker_size,
                fill=True,
                popup=f"Building allocated to First Aid Station with index: {i}",
                color=color,
            ).add_to(allocated_buildings_demand_fg)
folium.LayerControl().add_to(m)
m

Inspect the solutions

In [ ]:
solution